<a href="https://colab.research.google.com/github/saadahmedsh/Binary-and-multi-class-sarcasm-detection/blob/main/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, SimpleRNN
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import asarray
from numpy import zeros

In [ ]:
upload=files.upload()

Saving Sarcasm Dataset.csv to Sarcasm Dataset.csv


In [ ]:
df=pd.read_csv("Sarcasm Dataset.csv")

In [ ]:
df['tweet'] = df['tweet'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', str(s)))
stemmer = PorterStemmer()
df['tweet'] = df['tweet'].apply(lambda x: x.split())
df['tweet'] = df['tweet'].apply(lambda x : ' '.join([stemmer.stem(word) for word in x]))

In [ ]:
layers=[2,3]
dropout=[0.3 , 0.7]
layer_types = [SimpleRNN, LSTM, GRU]

In [ ]:
max_features = 2200
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['tweet'].values)
X = tokenizer.texts_to_sequences(df['tweet'].values)
X = pad_sequences(X)

In [ ]:
Y = pd.get_dummies(df['sarcastic']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.75, random_state = 42)


In [ ]:
df['sarcastic'].value_counts()

0    2601
1     867
Name: sarcastic, dtype: int64

In [ ]:
def score_predicted_values(dropout, layers, layer_type, output_neurons):
  dimensions = 58
  neurons = 196
  batch_size = 32

  model = Sequential()

  model.add(Embedding(max_features, dimensions,input_length = X.shape[1]))
 
  model.add(SpatialDropout1D(0.4))
  for i in range(0, layers-1):
    model.add(layer_type(neurons, dropout=dropout, recurrent_dropout=dropout, return_sequences=True))

  model.add(layer_type(neurons, dropout=dropout, recurrent_dropout=dropout))
  model.add(Dense(output_neurons,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
 
  lstm_train = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 0)
 
  values= model.predict(X_test)
  values[values> 0.5] = 1
  values[values <= 0.5] = 0
  print("......... Dropout :" + str(dropout) + " Layers: " + str(layers) + " Model type: " + str(layer_type) + " ........")
  print(".......................................................")
  print(classification_report(Y_test, values))

In [ ]:
print('........ Binary Classification .......')
for num_layers in layers:
  for drop in dropout:
    for _layer in layer_types:
      score_predicted_values(drop, num_layers, _layer, 2)

........ Binary Classification .......
......... Dropout :0.3 Layers: 2 Model type: <class 'keras.layers.recurrent.SimpleRNN'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      1951
           1       0.00      0.00      0.00       650

   micro avg       0.75      0.75      0.75      2601
   macro avg       0.38      0.50      0.43      2601
weighted avg       0.56      0.75      0.64      2601
 samples avg       0.75      0.75      0.75      2601



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.3 Layers: 2 Model type: <class 'keras.layers.recurrent_v2.LSTM'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.75      0.98      0.85      1951
           1       0.39      0.03      0.06       650

   micro avg       0.75      0.75      0.75      2601
   macro avg       0.57      0.51      0.46      2601
weighted avg       0.66      0.75      0.66      2601
 samples avg       0.75      0.75      0.75      2601

......... Dropout :0.3 Layers: 2 Model type: <class 'keras.layers.recurrent_v2.GRU'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.76      0.90      0.82      1951
           1       0.32      0.14      0.19       650

   micro avg       0.71      0.71      0.71      2601
   macro avg       0.54      0.52      0.51      2601
weighted avg       0.65      0.71      0.67   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.7 Layers: 2 Model type: <class 'keras.layers.recurrent_v2.LSTM'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.75      0.99      0.85      1951
           1       0.32      0.02      0.03       650

   micro avg       0.74      0.74      0.74      2601
   macro avg       0.53      0.50      0.44      2601
weighted avg       0.64      0.74      0.65      2601
 samples avg       0.74      0.74      0.74      2601

......... Dropout :0.7 Layers: 2 Model type: <class 'keras.layers.recurrent_v2.GRU'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      1951
           1       1.00      0.00      0.01       650

   micro avg       0.75      0.75      0.75      2601
   macro avg       0.88      0.50      0.43      2601
weighted avg       0.81      0.75      0.64   

...

......... Dropout :0.3 Layers: 3 Model type: <class 'keras.layers.recurrent_v2.LSTM'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.76      0.95      0.84      1951
           1       0.34      0.08      0.12       650

   micro avg       0.73      0.73      0.73      2601
   macro avg       0.55      0.51      0.48      2601
weighted avg       0.65      0.73      0.66      2601
 samples avg       0.73      0.73      0.73      2601

......... Dropout :0.3 Layers: 3 Model type: <class 'keras.layers.recurrent_v2.GRU'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.77      0.77      0.77      1951
           1       0.30      0.30      0.30       650

   micro avg       0.65      0.65      0.65      2601
   macro avg       0.53      0.53      0.53      2601
weighted avg       0.65      0.65      0.65   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.7 Layers: 3 Model type: <class 'keras.layers.recurrent_v2.LSTM'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      1951
           1       0.00      0.00      0.00       650

   micro avg       0.75      0.75      0.75      2601
   macro avg       0.38      0.50      0.43      2601
weighted avg       0.56      0.75      0.64      2601
 samples avg       0.75      0.75      0.75      2601



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.7 Layers: 3 Model type: <class 'keras.layers.recurrent_v2.GRU'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.75      0.98      0.85      1951
           1       0.26      0.02      0.03       650

   micro avg       0.74      0.74      0.74      2601
   macro avg       0.51      0.50      0.44      2601
weighted avg       0.63      0.74      0.65      2601
 samples avg       0.74      0.74      0.74      2601



In [ ]:
def score_predicted_values_bidirectional(dropout, layers, output_neurons):
  dimensions = 58
  neurons = 196
  batch_size = 32

  model = Sequential()
  model.add(Embedding(max_features, dimensions,input_length = X.shape[1]))
  
  model.add(SpatialDropout1D(0.4))
  for i in range(0, layers-1):
    model.add(Bidirectional(LSTM(neurons, return_sequences=True, recurrent_dropout=dropout, dropout=dropout)))
  model.add(Bidirectional(LSTM(neurons, recurrent_dropout=dropout, dropout=dropout)))
  model.add(Dense(output_neurons,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  lstm_train = model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 0)
  values= model.predict(X_test)
  values[values> 0.5] = 1
  values[values <= 0.5] = 0
  print("......... Dropout :" + str(dropout) + " Layers: " + str(layers) + " Model type: " + "Bidirectional" + " ........")
  print(".......................................................")
  print(classification_report(Y_test, values))

In [ ]:
for num_layers in layers:
  for drop in dropout:
    score_predicted_values_bidirectional(drop, num_layers, 2)

......... Dropout :0.3 Layers: 2 Model type: Bidirectional ........
.......................................................
              precision    recall  f1-score   support

           0       0.76      0.90      0.82      1951
           1       0.34      0.16      0.21       650

   micro avg       0.71      0.71      0.71      2601
   macro avg       0.55      0.53      0.52      2601
weighted avg       0.66      0.71      0.67      2601
 samples avg       0.71      0.71      0.71      2601

......... Dropout :0.7 Layers: 2 Model type: Bidirectional ........
.......................................................
              precision    recall  f1-score   support

           0       0.75      0.96      0.84      1951
           1       0.29      0.05      0.09       650

   micro avg       0.73      0.73      0.73      2601
   macro avg       0.52      0.50      0.46      2601
weighted avg       0.64      0.73      0.65      2601
 samples avg       0.73      0.73      0.73  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
sarcastic_df=df.copy()

In [ ]:
sarcastic_df['labels']=[0] * len(sarcastic_df)
columns=['rhetorical_question', 'overstatement', 'understatement',
       'satire', 'irony', 'sarcasm']
for i in range(0, len(columns)):
  sarcastic_df['labels'] = np.where((sarcastic_df[columns[i]] == 1),(len(columns) - i),sarcastic_df['labels'])

sarcastic_df=sarcastic_df.iloc[np.where(df['sarcastic'] == 1)].dropna()

In [ ]:
encoding=[[1,0,0,0,0,0], [0,1,0,0,0,0], [0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]

In [ ]:
labels=[]
features=sarcastic_df['tweet']
for index, row in sarcastic_df.iterrows():
  labels.append(encoding[row['labels'] - 1])


In [ ]:
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)
X = pad_sequences(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,labels, test_size = 0.25, random_state = 42)
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)

In [ ]:
print('........ Multi Classification .......')
for num_layers in layers:
  for drop in dropout:
    for _layer in layer_types:
      score_predicted_values(drop, num_layers, _layer, 6)

........ Multi Classification .......
......... Dropout :0.3 Layers: 2 Model type: <class 'keras.layers.recurrent.SimpleRNN'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       185
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.85      0.85      0.85       217
   macro avg       0.14      0.17      0.15       217
weighted avg       0.73      0.85      0.78       217
 samples avg       0.85      0.85      0.85       217



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.3 Layers: 2 Model type: <class 'keras.layers.recurrent_v2.LSTM'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       185
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.85      0.85      0.85       217
   macro avg       0.14      0.17      0.15       217
weighted avg       0.73      0.85      0.78       217
 samples avg       0.85      0.85      0.85       217



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.3 Layers: 2 Model type: <class 'keras.layers.recurrent_v2.GRU'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      0.96      0.90       185
           1       0.11      0.03      0.05        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.82      0.82      0.82       217
   macro avg       0.16      0.16      0.16       217
weighted avg       0.74      0.82      0.78       217
 samples avg       0.82      0.82      0.82       217



...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.7 Layers: 3 Model type: <class 'keras.layers.recurrent_v2.LSTM'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       185
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.85      0.85      0.85       217
   macro avg       0.14      0.17      0.15       217
weighted avg       0.73      0.85      0.78       217
 samples avg       0.85      0.85      0.85       217



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.7 Layers: 3 Model type: <class 'keras.layers.recurrent_v2.GRU'> ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       185
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.85      0.85      0.85       217
   macro avg       0.14      0.17      0.15       217
weighted avg       0.73      0.85      0.78       217
 samples avg       0.85      0.85      0.85       217



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for num_layers in layers:
  for drop in dropout:
    score_predicted_values_bidirectional(drop, num_layers, 6)

......... Dropout :0.3 Layers: 2 Model type: Bidirectional ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       185
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.85      0.85      0.85       217
   macro avg       0.14      0.17      0.15       217
weighted avg       0.73      0.85      0.78       217
 samples avg       0.85      0.85      0.85       217



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.7 Layers: 2 Model type: Bidirectional ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       185
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.85      0.85      0.85       217
   macro avg       0.14      0.17      0.15       217
weighted avg       0.73      0.85      0.78       217
 samples avg       0.85      0.85      0.85       217



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.3 Layers: 3 Model type: Bidirectional ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       185
           1       0.11      0.12      0.12        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.73      0.72      0.73       217
   macro avg       0.16      0.16      0.16       217
weighted avg       0.74      0.72      0.73       217
 samples avg       0.72      0.72      0.72       217



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


......... Dropout :0.7 Layers: 3 Model type: Bidirectional ........
.......................................................
              precision    recall  f1-score   support

           0       0.85      0.99      0.92       185
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0

   micro avg       0.85      0.84      0.85       217
   macro avg       0.14      0.16      0.15       217
weighted avg       0.73      0.84      0.78       217
 samples avg       0.84      0.84      0.84       217



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
################## Question 2 ########################
# GLOVE
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-05-09 04:32:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-05-09 04:32:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-05-09 04:32:27--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
tweets=df['tweet']
token = Tokenizer()
token.fit_on_texts(tweets)
vocab_size = len(token.word_index) + 1
encoded_tweet = token.texts_to_sequences(tweets)

In [ ]:
max_len=len(max(encoded_tweet, key=len))
padded_tweet = pad_sequences(encoded_tweet, maxlen=max_len, padding='post')

In [ ]:
embeddings_index_glove = dict()
f = open('glove.6B.50d.txt')
for line in f:
	values = line.split()

	word = values[0]  
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index_glove[word] = coefs
f.close()

In [ ]:
def generate_embedding_matrix(embed):
  embedding_matrix = zeros((vocab_size, 50))
  for word, i in token.word_index.items():
	  embedding_vector = embed.get(word)
	  if embedding_vector is not None:
		  embedding_matrix[i] = embedding_vector
  return embedding_matrix

  

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(padded_tweet,df['sarcastic'], test_size = 0.25, random_state = 42)

In [ ]:
def optimal_model(embedding_matrix, vocab, length):
  model = Sequential()
  embed = Embedding(vocab,length, weights=[embedding_matrix], input_length=max_len, trainable=False)
  model.add(embed)
  model.add(SimpleRNN(58, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
  model.add(SimpleRNN(58, dropout=0.3, recurrent_dropout=0.3))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  model.fit(X_train, Y_train, epochs=10, verbose=0)
  values= model.predict(X_test)
  values[values> 0.5] = 1
  values[values <= 0.5] = 0
  print("......... Dropout :" + str(0.3) + " Layers: " + str(2) + " Model type: " + str("RNN") + " ........")
  print(".......................................................")
  print(classification_report(Y_test, values))



In [ ]:
#Optimal Parameter with Glove
optimal_model(generate_embedding_matrix(embeddings_index_glove), vocab_size, 50)

......... Dropout :0.3 Layers: 2 Model type: RNN ........
.......................................................
              precision    recall  f1-score   support

           0       0.72      1.00      0.83       620
           1       0.00      0.00      0.00       247

    accuracy                           0.72       867
   macro avg       0.36      0.50      0.42       867
weighted avg       0.51      0.72      0.60       867



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Word2Vec
import gensim
from gensim.models import Word2Vec

In [ ]:
processedLines = [gensim.utils.simple_preprocess(sentence) for sentence in df['tweet']]
word_list = [word for words in processedLines for word in words]

In [ ]:
model = Word2Vec(
    [word_list],
    negative=10,
    iter=50,
    min_count=1,
    size=50 #dimension of the word vector
    )

In [ ]:
embeddings_index_word_vec = dict()
for word in word_list:
  embeddings_index_word_vec[word]=model[word]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Optimal Parameter with Word2Vec
optimal_model(generate_embedding_matrix(embeddings_index_word_vec), vocab_size, 50)

......... Dropout :0.3 Layers: 2 Model type: RNN ........
.......................................................
              precision    recall  f1-score   support

           0       0.72      1.00      0.83       620
           1       0.00      0.00      0.00       247

    accuracy                           0.72       867
   macro avg       0.36      0.50      0.42       867
weighted avg       0.51      0.72      0.60       867



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import nltk

In [ ]:
#Fasttext
from gensim.models.fasttext import FastText

In [ ]:
word_punctuation_tokenizer = nltk.WordPunctTokenizer()
word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in df['tweet']]

In [ ]:
embedding_size = 50
window_size = 40
min_word = 5
down_sampling = 1e-2

embeddings_fasttext = FastText(word_tokenized_corpus,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      workers = 4,
                      sg=1,
                      iter=100)

In [ ]:
zero_buffer=[0] * 50

In [ ]:
embeddings_index_fasttext = {}
i=0
for sentence in word_tokenized_corpus:
  for word in sentence:
    try:
      embeddings_index_fasttext[word]=embeddings_fasttext.wv[sentence][i]
    except:
       embeddings_index_fasttext[word]=zero_buffer
    i+=1
  i=0



In [ ]:
#optimal model for fasttext
optimal_model(generate_embedding_matrix(embeddings_index_fasttext), vocab_size, 50)

......... Dropout :0.3 Layers: 2 Model type: RNN ........
.......................................................
              precision    recall  f1-score   support

           0       0.72      1.00      0.83       620
           1       0.00      0.00      0.00       247

    accuracy                           0.72       867
   macro avg       0.36      0.50      0.42       867
weighted avg       0.51      0.72      0.60       867



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Elmo
import tensorflow_hub as hub

In [ ]:
elmo = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [ ]:
embeddings = elmo(df['tweet'].tolist())

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()



In [ ]:
embeddings=np.array(embeddings)

In [ ]:
def pad_zeros(arr, number):
  new_arr=arr.copy()
  for i in range(len(arr), number):
    new_arr.append(0)
  
  return np.array(new_arr)


In [ ]:
embed_buffer=[]
for i in range(0, len(embeddings)):
  try:
    embed_buffer.append(pad_zeros(embeddings[i].tolist(), len(df['tweet'][i].split(' ')) * 50).reshape(len(df['tweet'][i].split(' ')),50))
  except:
    # ignoring some sentences
   embed_buffer.append(embeddings[i][0:len(df['tweet'][i].split(' ')) * 50 ])

  

In [ ]:
embeddings_index_elmo=dict()
sentence_index=0
for sentence in df['tweet']:
  sentence_arr=sentence.split(' ')
  for index in (0, len(embed_buffer[sentence_index])):
    try:
      embeddings_index_elmo[sentence_arr[index]]=embed_buffer[sentence_index][index]
    except:
      print(sentence_index,index)
  sentence_index=+1
      


In [ ]:
# Optimal Model ELMO

optimal_model(generate_embedding_matrix(embeddings_index_elmo), vocab_size, 50)


......... Dropout :0.3 Layers: 2 Model type: RNN ........
.......................................................
              precision    recall  f1-score   support

           0       0.72      1.00      0.83       620
           1       0.00      0.00      0.00       247

    accuracy                           0.72       867
   macro avg       0.36      0.50      0.42       867
weighted avg       0.51      0.72      0.60       867



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
